In [45]:
import pandas as pd
import json

In [46]:
def save_fileInJSON(content ,strRoot : str, mode = 'w'):
    with open(strRoot,mode) as file:
        file.write(json.dumps(content))

In [47]:
def get_compressedData(listContent_One : list, listContent_Two : list, listHead : list):
    listVar = []
    for i in range(min(len(listContent_One), len(listContent_Two))):
        listVar.append( {listHead[0]:listContent_One[i], listHead[1]:listContent_Two[i]} )
    return listVar

In [48]:
data = pd.read_csv('data_vk.csv')

In [49]:
data = data.rename(columns={"Название площадки": "name", "Ссылка на сообщество": "link"
                    , "Количество участников": "subs", "Охват рекламы": "oxvat_rek"
                    , "Охват по подписчикам": "oxvat_subs", "Стоимость": "cost",
                     "Длительность размещения": "late_time"})

In [50]:
data =data.drop(['link','oxvat_rek','late_time'],axis=1)

In [51]:
data['predictCTR'] = (0.89/100)

In [52]:
data['CPC_predict'] = data['cost']/(data['predictCTR']*data['oxvat_subs'])

In [53]:
average_average_bank = 4000
ratio_effectiv = 0.05

In [54]:
data['predict_match_views'] = (data['oxvat_subs']*data['predictCTR']).astype(int)

In [55]:
data = data.sort_values(by=['predict_match_views'],ascending=False)

In [56]:
data['predict_match_users'] = (data['predict_match_views']*ratio_effectiv).astype(int)

In [57]:
data['predict_profit_bank'] = data['predict_match_users']*average_average_bank

In [58]:
data = data[data['predict_match_users']>0]

In [59]:
data['cost_one_user'] = (data['cost']/data['predict_match_users']).astype(int)

In [60]:
data['predict_profit_bank_prize'] = data['predict_profit_bank'] - data['cost']

In [61]:
data = data.sort_values(by=['predict_profit_bank_prize'],ascending=False)

In [62]:
data.index = data.name

Профит банка

In [63]:
save_fileInJSON(get_compressedData(data.predict_profit_bank_prize.index.tolist() ,  data.predict_profit_bank_prize.values.tolist(), ['PlatformName','PredictProfit']), 'JSON_RTG/bankProfit_VK.json')

FileNotFoundError: [Errno 2] No such file or directory: 'JSON_RTG/bankProfit_VK.json'

Стоимость одного клиента

In [ ]:
save_fileInJSON(get_compressedData(data.cost_one_user.index.tolist() ,  data.cost_one_user.values.tolist(), ['CountOfClient','PlatformName']), 'JSON_RTG/PricePerClient_VK.json')

Количество новых клиентов

In [ ]:
save_fileInJSON(get_compressedData(data.predict_match_users.index.tolist() ,  data.predict_match_users.values.tolist(), ['CountOfClient','PlatformName']), 'JSON_RTG/CountOfNewClients_VK.json')